In [1]:
%matplotlib inline
import os
import sys
import platform
import matplotlib
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as dates
import matplotlib.ticker as tick
from matplotlib.backends.backend_pdf import PdfPages
import datetime
from pylab import rcParams
rcParams['figure.figsize'] = 15, 10

In [3]:
#import loggerloader as ll
import wellapplication as wa
import arcpy

In [4]:
print("Operating System " + platform.system() + " " + platform.release())
print("Python Version " + str(sys.version))
print("Pandas Version " + str(pd.__version__))
print("Numpy Version " + str(np.__version__))
print("Matplotlib Version " + str(matplotlib.__version__))

Operating System Windows 10
Python Version 3.6.2 |Continuum Analytics, Inc.| (default, Jul 20 2017, 12:30:02) [MSC v.1900 64 bit (AMD64)]
Pandas Version 0.22.0
Numpy Version 1.13.3
Matplotlib Version 2.0.2


In [5]:
drive = 'G:'
raw_archive_folder = drive + '/My Drive/WORK/Snake Valley'
folder = raw_archive_folder + '/WaterMonitoring/PiezometerData/2017_2/Raw/'
enteredFolder = folder + '/entered/'
checkFolder = folder + '/toCheck/'
#wellinfofile = drive + raw_archive_folder + '/well table 2015-03-23.csv'

In [5]:
if not os.path.exists(enteredFolder):
    print('Creating Output Folder')
    os.makedirs(enteredFolder)
else:
    print('Output Folder Exists')

Output Folder Exists


In [6]:
if not os.path.exists(checkFolder):
    print('Creating Check Folder')
    os.makedirs(checkFolder)
else:
    print('Check Folder Exists')

Check Folder Exists


Inputs for connection file and tables

In [7]:
conn_file_root = "C:/Users/PAULINKENBRANDT/AppData/Roaming/ESRI/Desktop10.5/ArcCatalog/"
conn_file = "UGS_SDE.sde" #production
arcpy.env.workspace = conn_file_root + conn_file
gw_reading_table = "UGGP.UGGPADMIN.UGS_GW_reading"
station_table =  "UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Locations"

# Snake Valley Data Import

read over raw files -> append baro to db -> import well xle -> pull bp data from db -> remove bp press -> import manual measurements -> fix drift -> remove stickup -> include elevation

## Import Barometric Data into Database

In [ ]:
loc_table = "UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Locations"

# create empty dataframe to house well data
field_names = ['LocationID', 'LocationName', 'LocationType', 'LocationDesc', 'AltLocationID', 'Altitude',
               'AltitudeUnits', 'WellDepth', 'SiteID', 'Offset', 'LoggerType', 'BaroEfficiency',
               'BaroEfficiencyStart', 'BaroLoggerType']
df = pd.DataFrame(columns=field_names)
# populate dataframe with data from SDE well table
search_cursor = arcpy.da.SearchCursor(loc_table, field_names)
for row in search_cursor:
    # combine the field names and row items together, and append them
    df = df.append(dict(zip(field_names, row)), ignore_index=True)
df.dropna(subset=['AltLocationID'],inplace=True)
df

In [ ]:
xles = self.xle_head_table(self.xledir + '/')
arcpy.AddMessage('xles examined')
csvs = self.csv_info_table(self.xledir + '/')
arcpy.AddMessage('csvs examined')
file_info_table = pd.concat([xles, csvs[0]])


In [ ]:
def csv_info_table(self, folder):
    csv = {}
    files = [f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]
    field_names = ['filename', 'Start_time', 'Stop_time']
    df = pd.DataFrame(columns=field_names)
    for file in files:
        fileparts = os.path.basename(file).split('.')
        filetype = fileparts[1]
        basename = fileparts[0]
        if filetype == 'csv':
            try:
                cfile = {}
                csv[basename] = self.new_csv_imp(os.path.join(folder, file))
                cfile['Battery_level'] = int(round(csv[basename].loc[csv[basename]. \
                                                   index[-1], 'Volts'] / csv[basename]. \
                                                   loc[csv[basename].index[0], 'Volts'] * 100, 0))
                cfile['Sample_rate'] = (csv[basename].index[1] - csv[basename].index[0]).seconds * 100
                cfile['filename'] = basename
                cfile['fileroot'] = basename
                cfile['full_filepath'] = os.path.join(folder, file)
                cfile['Start_time'] = csv[basename].first_valid_index()
                cfile['Stop_time'] = csv[basename].last_valid_index()
                cfile['Location'] = ' '.join(basename.split(' ')[:-1])
                cfile['trans type'] = 'Global Water'
                df = df.append(cfile, ignore_index=True)
            except:
                pass
    df.set_index('filename', inplace=True)
    return df, csv

In [ ]:
files = ['pw20 20170307.xle','pw03z 20170309.xle','pw01c 20170308.csv']
file_extension = []
for file in files:
    file_extension.append(os.path.splitext(file)[1])
'.xle' in file_extension and '.csv' in file_extension


In [ ]:
files = ['pw20 20170307.xle','pw03z 20170309.xle','pw01c 20170308.csv']
    if 'xle' in file_extension:
            xles = self.xle_head_table(dirpath)
            arcpy.AddMessage('xles examined')
            file_info_table = xles
        if 'csv' in file_extension:
            csvs = self.csv_info_table(dirpath)
            arcpy.AddMessage('csvs examined')
            file_info_table = csvs

In [ ]:
file_info_table = pd.read_csv(r'M:\PROJECTS\Snake Valley Water\Transducer Data\Raw_data_archive\2017\file_info_table.csv')
maxtime = max(pd.to_datetime(file_info_table['Stop_time']))
mintime = min(pd.to_datetime(file_info_table['Start_time']))
#arcpy.AddMessage("Data span from {:} to {:}.".format(mintime,maxtime))

# upload barometric pressure data
baro_out = {}
#baros = well_table[well_table['LocationType'] == 'Barometer']
baros = []
if len(baros) < 1:
    baro_out['9003'] = get_location_data('9003', mintime, maxtime + datetime.timedelta(days=1))

In [ ]:
def get_location_data(site_number, first_date=None, last_date=None, limit=None,
                      gw_reading_table="UGGP.UGGPADMIN.UGS_GW_reading"):

    if not first_date:
        first_date = datetime.datetime(1900, 1, 1)
    elif type(first_date) == str:
        try:
            datetime.datetime.strptime(first_date, '%m/%d/%Y')
        except:
            first_date = datetime.datetime(1900, 1, 1)
    # Get last reading at the specified location
    if not last_date or last_date > datetime.datetime.now():
        last_date = datetime.datetime.now()
    query_txt = "LOCATIONID = '{:}' and (READINGDATE >= '{:%m/%d/%Y}' and READINGDATE <= '{:%m/%d/%Y}')"
    query = query_txt.format(site_number, first_date, last_date + datetime.timedelta(days=1))
    sql_sn = (limit, 'ORDER BY READINGDATE ASC')

    fieldnames = ll.get_field_names(gw_reading_table)
    print(fieldnames)
    readings = ll.table_to_pandas_dataframe(gw_reading_table, fieldnames, query, sql_sn)
    print(readings)
    readings.set_index('READINGDATE', inplace=True)
    if len(readings) == 0:
        arcpy.AddMessage('No Records for location {:}'.format(site_number))
    return readings

In [ ]:
baro_out['9003']

In [ ]:
shutil.rmtree(dirpath)

In [ ]:
ll.get_field_names(gw_reading_table)

In [ ]:
os.listdir(dirpath)

In [ ]:
wellimp.well_files

In [ ]:
xles = ll.xle_head_table(folder)
csvs = ll.csv_info_table(folder)
file_info_table = pd.concat([xles, csvs[0]])
file_info_table.columns

In [ ]:
wellid = '43'
station_table =  "UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Locations"
#arcpy.env.workspace = self.sde_conn
loc_table = "UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Locations"

field_names = ['LocationID', 'LocationName', 'LocationType', 'LocationDesc', 'AltLocationID', 'Altitude', 
               'AltitudeUnits', 'WellDepth', 'SiteID', 'Offset', 'LoggerType', 'BaroEfficiency', 
               'BaroEfficiencyStart', 'BaroLoggerType']
df = pd.DataFrame(columns=field_names)

# use a search cursor to iterate rows
search_cursor = arcpy.da.SearchCursor(loc_table, field_names)

    # iterate the rows
for row in search_cursor:
        # combine the field names and row items together, and append them
    df = df.append(dict(zip(field_names, row)), ignore_index=True)
    
iddict = dict(zip(df['LocationName'].values,df['AltLocationID'].values))

well_table = df.set_index(['AltLocationID'])
baroid = well_table.loc[wellid,'BaroLoggerType']
stickup = well_table.loc[wellid,'Offset']
well_elev = well_table.loc[wellid,'Altitude']
be = well_table.loc[wellid,'BaroEfficiency']


In [ ]:
['LocationID', 'LocationName', 'LocationType', 'LocationDesc', 'AltLocationID', 'Altitude', 'AltitudeUnits',
 'WellDepth', 'SiteID', 'Offset', 'LoggerType', 'BaroEfficiency', 'BaroEfficiencyStart', 'BaroLoggerType']

In [ ]:
fc = "UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Locations"
field = "LocationName"
cursor = arcpy.SearchCursor(fc)
ll.table_to_pandas_dataframe(fc, field_names=['LocationName''AltLocationID'])
for row in cursor:
    print(row.getValue(field))

In [ ]:
os.environ.get('USERNAME')

In [ ]:
os.path.splitext(well_file)[1] == '.xle'

In [ ]:
well_file = folder+'pw03z 20170309.xle'
baro_file = folder+'pw03baro 20170309.xle'
#ll.imp_one_well(well_file,baro_file,)

In [ ]:
well_table = ll.match_files_to_wellid(folder)
#query barometers to a single table

well_table.to_pickle(folder+'well_table.pickle')

In [ ]:
bpunits = []
baroid = well_table.loc[wellid,'BaroLoggerType']
stickup = well_table.loc[wellid,'Offset']
well_elev = well_table.loc[wellid,'Altitude']
be = well_table.loc[wellid,'BaroEfficiency']

## Run Transducer Processing

In [ ]:
well_table = ll.match_files_to_wellid(folder)
#query barometers to a single table

well_table.to_pickle(folder+'well_table.pickle')

In [ ]:
maxtime = max(pd.to_datetime(well_table['Stop_time']))
mintime = min(pd.to_datetime(well_table['Start_time']))
print('Pulling Barometric Pressure data from {:} to {:}'.format(mintime, maxtime))

In [ ]:
bpunits = []
for ind in well_table.index:
    if 'baro' in str(ind) or 'baro' in str(well_table.loc[ind,'Location']):
        bpunits.append(well_table.loc[ind,'WellID'])

baro_out = {}
for baroid in bpunits:
    baro_out[baroid] = ll.get_location_data(gw_reading_table, baroid, mintime, 
                                   maxtime + datetime.timedelta(days=1))
    baro_out[baroid].to_pickle(folder+str(baroid)+'.pickle')


## Bring in existing data

In [ ]:
well_table =pd.read_pickle(folder+'well_table.pickle')

bpunits = []
for ind in well_table.index:
    if 'baro' in str(ind) or 'baro' in str(well_table.loc[ind,'Location']):
        bpunits.append(well_table.loc[ind,'WellID'])

baro_out = {}
for baroid in bpunits:
    baro_out[baroid] = pd.read_pickle(folder + str(baroid) + '.pickle')

In [ ]:
manualwls = raw_archive_folder + '/All tape measurements.csv'
manual = pd.read_csv(manualwls, index_col="DateTime", engine="python")

In [ ]:
man_startdate = '1/1/2001' 
man_endate = '10/11/2002'
man_start_level = 10 
man_end_level =15.1

arcpy.AddMessage('Well {:} successfully imported!'.format(ind))

In [ ]:
indexedlist = [3]
indexedlist[0]


In [ ]:
well = ll.new_trans_imp(r'M:\PROJECTS\Snake Valley Water\Transducer Data\Raw_data_archive\2017subset\ag13a 20170308.xle')

In [ ]:
manualfile

In [ ]:

manl = pd.read_csv(u'M:\PROJECTS\Snake Valley Water\Transducer Data\Raw_data_archive\All tape measurements.csv')
manualfile = manl[manl['Location ID'] == int('71')]
manualfile.index = pd.to_datetime(manualfile.index)
manualfile.sort_index(inplace=True)
for i in range(len(manualfile)):
    print(wa.fcl(well, manualfile.index[i]).name)

## Upload Well Data To SDE

In [ ]:
barocolumn='MEASUREDLEVEL'
dft_ln = {}
dft_st = {}

from matplotlib.backends.backend_pdf import PdfPages
pdf_pages = PdfPages(folder + '/wells.pdf')

welltest = well_table.index.values
for ind in welltest:
    # import well file
    df, man, be, drift = ll.imp_well(well_table,ind,manual,baro_out)
    
    # plot data
    y1 = df['WATERELEVATION'].values
    y2 = df['barometer'].values
    x1 = df.index.values
    x2 = df.index.values

    x4 = man.index
    y4 = man['Meas_GW_Elev']
    fig, ax1 = plt.subplots()
    ax1.scatter(x4,y4,color='purple')
    ax1.plot(x1,y1,color='blue',label='Water Level Elevation')
    ax1.set_ylabel('Water Level Elevation',color='blue')
    ax1.set_ylim(min(df['WATERELEVATION']),max(df['WATERELEVATION']))
    y_formatter = tick.ScalarFormatter(useOffset=False)
    ax1.yaxis.set_major_formatter(y_formatter)
    ax2 = ax1.twinx()
    ax2.set_ylabel('Barometric Pressure (ft)', color='red') 
    ax2.plot(x2,y2,color='red',label='Barometric pressure (ft)')
    h1, l1 = ax1.get_legend_handles_labels()
    h2, l2 = ax2.get_legend_handles_labels()
    ax1.legend(h1+h2, l1+l2, loc=3)
    plt.xlim(df.first_valid_index()-datetime.timedelta(days=3),df.last_valid_index()+datetime.timedelta(days=3))
    plt.title('Well: {:}  Drift: {:}  Baro. Eff.: {:}'.format(ind,drift,be))

    pdf_pages.savefig(fig)
    plt.close()



    
pdf_pages.close()
print("DONE!")
print("Files in "+ folder+'\\wells.pdf')

In [ ]:
pdf_pages.savefig(fig)
plt.close()



    
pdf_pages.close()
print("DONE!")
print("Files in "+ folder+'\\wells.pdf')

In [ ]:
def baro_eff(df, bp, wl, lag=200):
    import statsmodels.tsa.tsatools as tools
    df.dropna(inplace=True)
    dwl = df[wl].diff().values[1:-1]
    dbp = df[bp].diff().values[1:-1]
    # dwl = df[wl].values[1:-1]
    # dbp = df[bp].values[1:-1]
    df['j_dates'] = df.index.to_julian_date()
    lag_time = df['j_dates'].diff().cumsum().values[1:-1]
    df.drop('j_dates', axis=1, inplace=True)
    # Calculate BP Response Function

    ## create lag matrix for regression
    bpmat = tools.lagmat(dbp, lag, original='in')
    ## transpose matrix to determine required length
    ## run least squared regression
    sqrd = np.linalg.lstsq(bpmat, dwl)
    wlls = sqrd[0]
    cumls = np.cumsum(wlls)
    negcumls = [-1 * cumls[i] for i in range(len(cumls))]
    ymod = np.dot(bpmat, wlls)

    ## resid gives the residual of the bp
    resid = [(dwl[i] - ymod[i]) for i in range(len(dwl))]
    lag_trim = lag_time[0:len(cumls)]
    return negcumls, cumls, ymod, resid, lag_time, dwl, dbp, wlls

In [ ]:
baro_eff(df,'barometer','Level')

In [ ]:
# plot data
wl = 'cor2'
y1 = df[wl].values
y2 = df['barometer'].values
x1 = df.index.values
x2 = df.index.values

x4 = man.index
y4 = man['Meas_GW_Elev']
fig, ax1 = plt.subplots()
ax1.scatter(x4,y4,color='purple')
ax1.plot(x1,y1,color='blue',label='Water Level Elevation')
ax1.set_ylabel('Water Level Elevation',color='blue')
ax1.set_ylim(min(df[wl]),max(df[wl]))
y_formatter = tick.ScalarFormatter(useOffset=False)
ax1.yaxis.set_major_formatter(y_formatter)
ax2 = ax1.twinx()
ax2.set_ylabel('Barometric Pressure (ft)', color='red') 
ax2.plot(x2,y2,color='red',label='Barometric pressure (ft)')
h1, l1 = ax1.get_legend_handles_labels()
h2, l2 = ax2.get_legend_handles_labels()
ax1.legend(h1+h2, l1+l2, loc=3)
plt.xlim(df.first_valid_index()-datetime.timedelta(days=3),df.last_valid_index()+datetime.timedelta(days=3))
plt.title('Well: {:}  Drift: {:}  Baro. Eff.: {:}'.format(ind,drift,be))

In [ ]:
df['cor2'] = df[['Level', 'barometer']].\
        apply(lambda x: x[0] + 0.25 * (x[1]), 1)

In [ ]:
df['barometer'][0]

In [ ]:
plt.scatter(df['Level'].diff(), df['barometer'].diff())
import statsmodels.api as sm
df['dbp'] = df['barometer'].diff()
df['dwl'] = df['corrwl'].diff()
df1 = df.dropna(subset=['dbp','dwl'])
x = df1['dbp']
y = df1['dwl']
X = sm.add_constant(x)
model = sm.OLS(y, X).fit()
    # y_reg = [data.ix[i,'Sbp']*m+b for i in range(len(data['Sbp']))]
b = model.params[0]
m = model.params[1]
r = model.rsquared
print(m,r)

In [ ]:
be, intc, r = ll.clarks(df[500:600],'barometer','corrwl')
print(be,intc,r)

## Barometric Pressure

In [ ]:
fold2014 = "G:/My Drive/WORK/Snake Valley/WaterMonitoring/PiezometerData/2014_2/Raw/"
baro201402file = fold2014 + '1044548_2014_06_16.xle'
baro2014a = barofileimp(baro201402file, 9024)

In [ ]:
addbaro = "G:/My Drive/WORK/Snake Valley/Twini_Springs_Baro_9024.csv"
baro2014 = pd.read_csv(addbaro,index_col='READINGDATE')

In [ ]:
baro = pd.concat([baro, baro2014])
baro.drop_duplicates(inplace=True)

14	Nephi Municipal Airport Weather Station KU14<br>
<br>
9027	PW10<br>
9049	Barometer<br>
<br>
BARO3	1038964	Garrison PW03	9003<br>
BARO2	1044788	Twin Spring (Baro2)	9024<br>
BARO1	1044779	Lelland-Harris	9025<br>
pw19baro	1034820
pw10baro	1028270

In [ ]:
def compilation(inputfile):
    """This function reads multiple xle transducer files in a directory and generates a compiled Pandas DataFrame.
    Args:
        inputfile (file):
            complete file path to input files; use * for wildcard in file name
    Returns:
        outfile (object):
            Pandas DataFrame of compiled data
    Example::
        >>> compilation('O:\\Snake Valley Water\\Transducer Data\\Raw_data_archive\\all\\LEV\\*baro*')
        picks any file containing 'baro'
    """

    # create empty dictionary to hold DataFrames
    f = {}

    # generate list of relevant files
    filelist = glob.glob(inputfile)

    # iterate through list of relevant files
    for infile in filelist:
        print(infile)
        # get the extension of the input file
        filetype = os.path.splitext(infile)[1]
        # run computations using lev files
        if filetype == '.lev':
            # open text file
            try:
                with open(infile) as fd:
                    # find beginning of data
                    indices = fd.readlines().index('[Data]\n')

                # convert data to pandas dataframe starting at the indexed data line
                f[wa.getfilename(infile)] = pd.read_table(infile, parse_dates=True, sep='\s+', index_col=0,
                                                       skiprows=indices + 2,
                                                       names=['DateTime', 'Level', 'Temperature'],
                                                       skipfooter=1, engine='python')
                # add extension-free file name to dataframe
                f[wa.getfilename(infile)]['name'] = wa.getfilename(infile)
                f[wa.getfilename(infile)]['Level'] = pd.to_numeric(f[wa.getfilename(infile)]['Level'])
                f[wa.getfilename(infile)]['Temperature'] = pd.to_numeric(f[wa.getfilename(infile)]['Temperature'])
            except ValueError:
                pass
        elif filetype == '.xle':  # run computations using xle files
            try:
                f[wa.getfilename(infile)] = wa.new_xle_imp(infile)
            except ValueError:
                pass
        elif filetype == '.csv':
            try:
                f[wa.getfilename(infile)] = pd.read_csv(infile, parse_dates=0, index_col=0)
            except:
                pass
        else:

            pass
    # concatenate all of the DataFrames in dictionary f to one DataFrame: g
    g = pd.concat(f)
    try:
        g = g.reset_index()
        g = g.set_index(['DateTime'])
    except ValueError:
        g.drop(['DateTime'],axis=1,inplace=True)
        g = g.reset_index()
        g = g.set_index(['DateTime'])
    # drop old indexes
    g = g.drop(['level_0'], axis=1)
    # remove duplicates based on index then sort by index
    g['ind'] = g.index
    g.drop_duplicates(subset='ind', inplace=True)
    g.drop('ind', axis=1, inplace=True)
    g = g.sort_index()
    outfile = g
    return outfile

### Find and compile existing baro data

Search All files for Barometer Raw files and copy to a master barometer folder

In [ ]:
from shutil import copyfile
baronames = ['1038964','1044788','1044779','pw03baro','pw19baro',
             '1034820','pw10baro','1028270',
            'pw03 baro','pw19 baro','pw10 baro']
dir = "G:/My Drive/WORK/Snake Valley/Transducer Data/"
for pack in os.walk(dir):
    #print(pack[0])
    for baroname in baronames:
        for i in glob.glob(pack[0]+'/'+'*{:}*'.format(baroname)):
            if i[-4:] in ['.lev','.xle']:
                rightfile = str(os.path.getmtime(i))+"_"+os.path.basename(i)
                print(str(os.path.getmtime(i))+"_"+os.path.basename(i))
                baro = "G:/My Drive/WORK/Snake Valley/Barometers/"
                try:
                    copyfile(i, os.path.join(baro, rightfile))
                except:
                    pass
    #file_extension.append(os.path.splitext(file)[1])

Compile barometers in list

In [ ]:
bro = {}
for baroname in baronames:
    bro[baroname] = compilation(baro+'/*{:}*'.format(baroname))

In [ ]:
pw3fill = pd.read_csv('G:/My Drive/WORK/Snake Valley/Barometers/pw03baro_2012.csv',index_col=0,parse_dates=True)

In [ ]:
bro['pw03'] = pd.concat([bro['pw03baro'],bro['pw03 baro'],bro['1038964'],pw3fill])
bro['pw03'].sort_index(inplace=True)
bro['pw03'].drop_duplicates(inplace=True)
bro['pw03'] = bro['pw03'][bro['pw03'].index > pd.datetime(2009,6,9)] 
bro['pw03']['Level'].plot()
bro['pw03'].tail()
bro['pw03'].to_csv(baro+'pw03.csv')

In [ ]:
bro['pw10'] = pd.concat([bro['pw10baro'],bro['pw10 baro'],bro['1028270']])
bro['pw10'].drop_duplicates(inplace=True)
bro['pw10'] = bro['pw10'][bro['pw10'].index > pd.datetime(2009,6,9)] 
bro['pw10']['Level'].plot()
bro['pw10'].tail()
bro['pw10'].to_csv(baro+'pw10.csv')

In [ ]:
bro['pw19'] = pd.concat([bro['pw19baro'],bro['pw19 baro'],bro['1034820']])
bro['pw19'].drop_duplicates(inplace=True)
bro['pw19']['Level'].plot()
bro['pw19'].tail()
bro['pw19'].to_csv(baro+'pw19.csv')

In [ ]:
bro['1044779'].drop_duplicates(inplace=True)
bro['1044779']['Level'].plot()
bro['1044779'].tail()
bro['1044779'].to_csv(baro+'1044779.csv')

In [ ]:
bro['1044788'].drop_duplicates(inplace=True)
bro['1044788']['Level'].plot()
bro['1044788'].tail()
bro['1044788'].to_csv(baro+'1044788.csv')

### Read compiled data and upload new data

In [11]:
def barofileimp(df,altid):
    #df = ll.new_trans_imp(bfile)
    if 'name' in df.columns.values:
        df.drop(['name'],axis=1,inplace=True)
    if 'none' in df.columns.values:
        df.drop(['none'],axis=1,inplace=True)
    df.rename(columns = {'Temperature':'TEMP', 'Level':'MEASUREDLEVEL'},inplace=True)
    df.index.name = 'READINGDATE'
    df['TAPE'] = 0
    df['LOCATIONID'] = altid
    return df

In [8]:
saved_csvs = ['pw03','1044788','1044779','pw19','pw10']

barometerids = {'pw03':9003,'1044788':9024,'1044779':9025,'pw10':9027,'pw19':9049,'sg25':9061}
barodict = {"PW10 Barometer":9027, "PW19 Barometer":9049, "SG25 Barometer":9061, 
            "Leland-Harris Barometer":9025, "Twin Springs Barometer":9024, "PW03 Barometer":9003}
bpdict = {'pw03':'9003','pw10':'9027','pw19':'9049','twin':'9024','leland':'9025'}       

barodata = {}

barodrive = 'G:/My Drive/WORK/Snake Valley/Barometers/'

head = ['MEASUREDLEVEL', 'TEMP', 'LOCATIONID']
ind_head = ['READINGDATE']
for csv in saved_csvs:
    barodata[csv] = pd.read_csv(barodrive + '{:}.csv'.format(csv),
                                index_col=0,parse_dates=True)
    barodata[csv].sort_index(inplace=True)

In [ ]:

pw03baro_append = folder + "\\pw03 baro 2016-08-03.xle"
pw10baro_append = folder + "\\pw10 baro 2016-08-03.xle"
pw19baro_append = folder + "\\pw19 baro 2016-08-04.xle"

df = ll.new_trans_imp(bfile)
barofileimp(bfile,altid)
wa.appendomatic(pw03baro_append,pw03baro)

In [14]:
for csv in saved_csvs:
    print(csv)
    df = barofileimp(barodata[csv], barometerids[csv])
    maxinfo = ll.find_extreme(barometerids[csv])
    if maxinfo[0] < df.index.max():
        print(maxinfo,df.index.max())
        
        df = df[df.index > maxinfo[0]]#.to_csv(barodrive + '{:}_to_import.csv'.format(barometerids[csv]))
        df.reset_index(inplace=True)
        ll.edit_table(df, gw_reading_table, df.columns)                                                                     

pw03
1044788
1044779
pw19
pw10


In [ ]:
baro.to_csv(raw_archive_folder + '\\baro.csv')

In [ ]:
'ReadingID',
 'WellID',
 'DateTime',
 'MeasuredLevel',
 'Temp',

## Water Level Tranducer Data

### Export and Plot Data

#### Export Manual Data

In [ ]:
manualwls = raw_archive_folder + "\\All tape measurements.csv"
manual = pd.read_csv(manualwls, index_col="DateTime", engine="python")
manualrecent = manual[manual.index.to_datetime() > pd.datetime(2015,6,1)]
manualrecent.dropna(inplace=True)
manualrecent.reset_index(inplace=True)
print manualrecent.dtypes
manualrecent = pd.merge(manualrecent, wellinfo, how='left',left_on='WellID', right_index=True)
manualrecent.loc[:,"MeasuredLevel"] = np.nan
manualrecent.loc[:,"Temp"] = np.nan
manualrecent.loc[:,"BaroEfficiencyCorrected"] = np.nan 
manualrecent.loc[:,"DeltaLevel"] = np.nan
manualrecent.loc[:,"DriftCorrection"] = np.nan
manualrecent.loc[:,"MeasuredBy"] = np.nan
manualrecent.loc[:,"Tape"] = 1
manualrecent.loc[:,"DTWBelowGroundSurface"] = np.nan
manualrecent.loc[:,"WaterElevation"] = np.nan
#manualrecent["DTWBelowGroundSurface"] = np.nan
manualrecent.loc[:,"DTWBelowCasing"] = manualrecent.loc[:,"MeasuredDTW"]
manualrecent.loc[:,"DTWBelowGroundSurface"] = manualrecent.loc[:,"MeasuredDTW"] - manualrecent.loc[:,"Offset"]
manualrecent.loc[:,"WaterElevation"] = manualrecent.loc[:,'GroundElevation'] - manualrecent.loc[:,"DTWBelowGroundSurface"]
print manualrecent

#outpath = pathlist[0] + '\\' + pathlist[1] + '\\' + pathlist[2] + '\\' + pathlist[3] + '\\' + pathlist[4] + '\\' + 'Manual' + '.csv'  
manualrecent.to_csv(raw_archive_folder+ 'Manual' + '.csv', index=True, columns= ["WellID","DateTime","MeasuredLevel","Temp","BaroEfficiencyCorrected","DeltaLevel",
                                             "MeasuredDTW","DriftCorrection","DTWBelowCasing","DTWBelowGroundSurface",
                                             "WaterElevation","Tape","MeasuredBy"])

In [ ]:
manual['DateTime'] = manual.index.to_datetime()
manual.to_csv(raw_archive_folder+ 'Manual' + '.csv', index=False, columns= ["WellID","DateTime","MeasuredLevel","Temp","BaroEfficiencyCorrected","DeltaLevel",
                                             "MeasuredDTW","DriftCorrection","DTWBelowCasing","DTWBelowGroundSurface",
                                             "WaterElevation","Tape","MeasuredBy"])

#### List Files to Export

In [ ]:
print wellinfo.loc[:,'full_file_name']

## Files To Database

In [ ]:
manualwls = raw_archive_folder + '/All tape measurements.csv'
manual = pd.read_csv(manualwls, index_col="DateTime", engine="python")
barofile = raw_archive_folder + '/baro.csv'
baro = pd.read_csv(barofile,index_col=0, parse_dates=True)
wellinfo = pd.read_csv(folder + '/wellinfo4.csv')

In [ ]:
engine = engineGetter.getEngine()

### Add Select Wells

In [15]:


form = widgets.VBox()
welllist = ["pw07mx 2015-11-30.csv","pw02a 11-30-2015.xle","pw02a 2016-07-07.xle","sg23b 2016-05-02.xle"]
welllist = welllist+ list(wellinfo.loc[:,'full_file_name'].values)
#print welllist

wells = widgets.SelectMultiple(description="Well", options=welllist, padding=4)
pdfName = widgets.Text(description="PDF:",padding=4)
driftTol = widgets.FloatSlider(value=0.05, min=0.00, max=10.0, step=0.05, description='Drift Tolerance:')
form.children = [wells, pdfName, driftTol]
display(form)

NameError: name 'widgets' is not defined

In [ ]:
for i in wells.value:
    print folder+'/'+i
    inputfile = folder +'/'+i
    g, drift, wellname = svdi.imp_new_well(inputfile, wellinfo, manual, baro)
    glist = g.columns.tolist()
    y1 = g['WaterElevation'].values
    y2 = baro['pw03'].values
    x1 = g.index.values
    x2 = baro.index.values
    wellname, wellid = svdi.getwellid(folder+'\\'+i,wellinfo)
    ylast = wellinfo[wellinfo['WellID']==wellid]['GroundElevation'].values[0] + wellinfo[wellinfo['WellID']==wellid]['Offset'].values[0] - svdi.fcl(manual[manual['WellID']== wellid],max(g.index.to_datetime()))[1]
    yfirst = wellinfo[wellinfo['WellID']==wellid]['GroundElevation'].values[0] + wellinfo[wellinfo['WellID']==wellid]['Offset'].values[0] - svdi.fcl(manual[manual['WellID']== wellid],min(g.index.to_datetime()))[1]
    xlast = pd.to_datetime((svdi.fcl(manual[manual['WellID']== wellid],max(pd.to_datetime(g.index)))).name)
    xfirst = pd.to_datetime((svdi.fcl(manual[manual['WellID']== wellid],min(pd.to_datetime(g.index)))).name)
    x4 = [xfirst,xlast]
    y4 = [yfirst,ylast]
    fig, ax1 = plt.subplots()
    ax1.scatter(x4,y4,color='purple')
    ax1.plot(x1,y1,color='blue',label='Water Level Elevation')
    ax1.set_ylabel('Water Level Elevation',color='blue')
    y_formatter = tick.ScalarFormatter(useOffset=False)
    ax1.yaxis.set_major_formatter(y_formatter)
    ax2 = ax1.twinx()
    ax2.set_ylabel('Barometric Pressure (ft)', color='red') 
    ax2.plot(x2,y2,color='red',label='Barometric pressure (ft)')
    h1, l1 = ax1.get_legend_handles_labels()
    h2, l2 = ax2.get_legend_handles_labels()
    ax1.legend(h1+h2, l1+l2, loc=3)
    plt.xlim(xfirst-timedelta(days=3),xlast+timedelta(days=3))
    plt.title('Well: ' + wellname.title() + '  ' + 'Total Drift = ' + str(g['DriftCorrection'][-1]))

In [ ]:
env.workspace = "C:/Users/PAULINKENBRANDT/AppData/Roaming/ESRI/Desktop10.4/ArcCatalog/UEMP_Dev.sde"
read_table = "UEMP_Dev.UEMPADMIN.GW_reading"

arcpy.env.overwriteOutput=True
edit = arcpy.da.Editor(env.workspace)
edit.startEditing(False, True)
edit.startOperation()

In [ ]:
pdf_pages = PdfPages(folder + '/' + pdfName.value + '.pdf')
for i in wells.value:
    print folder+'/'+i
    inputfile = folder +'/'+i
    g, drift, wellname = svdi.imp_new_well(inputfile, wellinfo, manual, baro)
    quer = "SELECT * FROM groundwater.reading where WellID = " + str(g['WellID'].values[0]) + " and DateTime > \'" + str(g.index.values[-1])[0:10] + " "+ str(g.index.values[-1])[11:19] + "\'"

    if abs(float(drift)) < driftTol.value:
        if len(pd.read_sql_query(sql=quer,con=engine))<1:
            g.to_csv(enteredFolder+wellname+".csv", index=False)
            tablename = 'reading'
            g.to_sql(con=engine, name = tablename, if_exists='append', flavor='mysql', index=False)
            print("Added to DB table " + tablename)
        else:
            print("Already Entered")
            print(len(pd.read_sql_query(sql=quer,con=engine)))
    else:
        g.to_csv(checkFolder+wellname+".csv", index=False, columns= ["WellID","DateTime","MeasuredLevel","Temp","BaroEfficiencyCorrected","DeltaLevel",
                                             "MeasuredDTW","DriftCorrection","DTWBelowCasing","DTWBelowGroundSurface",
                                             "WaterElevation","Tape","MeasuredBy"])
        print("Check File")
    glist = g.columns.tolist()
    for j in range(len(glist)):
        if 'pw' in glist[j]:
            h = glist[j]
    y1 = g['WaterElevation'].values
    y2 = baro['pw03'].values
    x1 = g.index.values
    x2 = baro.index.values
    wellname, wellid = svdi.getwellid(folder+'\\'+i,wellinfo)
    ylast = wellinfo[wellinfo['WellID']==wellid]['GroundElevation'].values[0] + wellinfo[wellinfo['WellID']==wellid]['Offset'].values[0] - svdi.fcl(manual[manual['WellID']== wellid],max(pd.to_datetime(g.index)))[1]
    yfirst = wellinfo[wellinfo['WellID']==wellid]['GroundElevation'].values[0] + wellinfo[wellinfo['WellID']==wellid]['Offset'].values[0] - svdi.fcl(manual[manual['WellID']== wellid],min(pd.to_datetime(g.index)))[1]
    xlast = pd.to_datetime(svdi.fcl(manual[manual['WellID']== wellid],max(pd.to_datetime(g.index))).name)
    xfirst = pd.to_datetime(svdi.fcl(manual[manual['WellID']== wellid],min(pd.to_datetime(g.index))).name)
    x4 = [xfirst,xlast]
    y4 = [yfirst,ylast]
    fig, ax1 = plt.subplots()
    ax1.scatter(x4,y4,color='purple')
    ax1.plot(x1,y1,color='blue',label='Water Level Elevation')
    ax1.set_ylabel('Water Level Elevation',color='blue')
    y_formatter = tick.ScalarFormatter(useOffset=False)
    ax1.yaxis.set_major_formatter(y_formatter)
    ax2 = ax1.twinx()
    ax2.set_ylabel('Barometric Pressure (ft)', color='red') 
    ax2.plot(x2,y2,color='red',label='Barometric pressure (ft)')
    h1, l1 = ax1.get_legend_handles_labels()
    h2, l2 = ax2.get_legend_handles_labels()
    ax1.legend(h1+h2, l1+l2, loc=3)
    plt.xlim(xfirst-timedelta(days=3),xlast+timedelta(days=3))
    plt.title('Well: ' + wellname.title() + '  ' + 'Total Drift = ' + str(g['DriftCorrection'][-1]))
    
    pdf_pages.savefig(fig)
    plt.close()
pdf_pages.close()
print("DONE!")
print("Files in "+ folder+'\\wells.pdf')
print(datetime.now())

In [ ]:
form = widgets.VBox()
driftTol = widgets.FloatSlider(value=0.05, min=0.00, max=10.0, step=0.05, description='Drift Tolerance:')
form.children = [driftTol]
display(form)

In [ ]:
pdf_pages = PdfPages(folder + '/wells.pdf')
for i in wellinfo.loc[:,'full_file_name']:
    print folder+'/'+i
    inputfile = folder +'/'+i
    g, drift, wellname = svdi.imp_new_well(inputfile, wellinfo, manual, baro)
    quer = "SELECT * FROM groundwater.reading where WellID = " + str(g['WellID'].values[0]) + " and (DateTime >= \'" + str(g.index.values[-1])[0:10] + "\')"

    if abs(float(drift)) < driftTol.value:
        if len(pd.read_sql_query(sql=quer,con=engine))<1:
            g.to_csv(enteredFolder + wellname+".csv", index=False)
            g.to_sql(con=engine, name='reading', if_exists='append', flavor='mysql', index=False)
            print("Added to DB")
        else:
            print("Already Entered")
    else:
        g.to_csv(checkFolder + wellname+".csv", index=False, columns= ["WellID","DateTime","MeasuredLevel","Temp","BaroEfficiencyCorrected","DeltaLevel",
                                             "MeasuredDTW","DriftCorrection","DTWBelowCasing","DTWBelowGroundSurface",
                                             "WaterElevation","Tape","MeasuredBy"])
        print("Check File")
    glist = g.columns.tolist()
    for j in range(len(glist)):
        if 'pw' in glist[j]:
            h = glist[j]
    y1 = g['WaterElevation'].values
    y2 = baro['pw03'].values
    x1 = g.index.values
    x2 = baro.index.values
    wellname, wellid = svdi.getwellid(folder+'\\'+i,wellinfo)
    ylast = wellinfo[wellinfo['WellID']==wellid]['GroundElevation'].values[0] + wellinfo[wellinfo['WellID']==wellid]['Offset'].values[0] - svdi.fcl(manual[manual['WellID']== wellid],max(pd.to_datetime(g.index)))[1]
    yfirst = wellinfo[wellinfo['WellID']==wellid]['GroundElevation'].values[0] + wellinfo[wellinfo['WellID']==wellid]['Offset'].values[0] - svdi.fcl(manual[manual['WellID']== wellid],min(pd.to_datetime(g.index)))[1]
    xlast = pd.to_datetime(svdi.fcl(manual[manual['WellID']== wellid],max(pd.to_datetime(g.index))).name)
    xfirst = pd.to_datetime(svdi.fcl(manual[manual['WellID']== wellid],min(pd.to_datetime(g.index))).name)
    x4 = [xfirst,xlast]
    y4 = [yfirst,ylast]
    fig, ax1 = plt.subplots()
    ax1.scatter(x4,y4,color='purple')
    ax1.plot(x1,y1,color='blue',label='Water Level Elevation')
    ax1.set_ylabel('Water Level Elevation',color='blue')
    y_formatter = tick.ScalarFormatter(useOffset=False)
    ax1.yaxis.set_major_formatter(y_formatter)
    ax2 = ax1.twinx()
    ax2.set_ylabel('Barometric Pressure (ft)', color='red') 
    ax2.plot(x2,y2,color='red',label='Barometric pressure (ft)')
    h1, l1 = ax1.get_legend_handles_labels()
    h2, l2 = ax2.get_legend_handles_labels()
    ax1.legend(h1+h2, l1+l2, loc=3)
    plt.xlim(min(pd.to_datetime(g.index))-timedelta(days=3),max(pd.to_datetime(g.index))+timedelta(days=3))
    plt.title('Well: ' + wellname.title() + '  ' + 'Total Drift = ' + str(g['DriftCorrection'][-1]))
    
    pdf_pages.savefig(fig)
    plt.close()
pdf_pages.close()
print("DONE!")
print("Files in "+ folder+'\\wells.pdf')

# Revised Workflow

In [6]:
conn_file_root = "C:/Users/PAULINKENBRANDT/AppData/Roaming/ESRI/Desktop10.5/ArcCatalog/"
conn_file = "UGS_SDE.sde" #production
arcpy.env.workspace = conn_file_root + conn_file
gw_reading_table = "UGGP.UGGPADMIN.UGS_GW_reading"
stations_table =  "UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Locations"
well_table = ll.table_to_pandas_dataframe(stations_table)

well_table.set_index('AltLocationID',inplace=True)
well_table = ll.table_to_pandas_dataframe(stations_table)
well_table.set_index('AltLocationID',inplace=True)
well_table.loc['7001', 'BaroLoggerType']

'9062'

In [7]:
manual_file = 'G:/My Drive/WORK/Snake Valley/manual_measurements.csv'
manual = pd.read_csv(manual_file, index_col="DateTime")

In [18]:
well_table['BaroLoggerType'].unique()

array(['9061', '9003', None, '9049', '9027', '9062'], dtype=object)

In [19]:
query = "LOCATIONID in('9061', '9003', '9049', '9027', '9062')"
baro_out = ll.table_to_pandas_dataframe(gw_reading_table,query=query)

In [17]:
baro_out = {}
for baro in well_table['BaroLoggerType'].unique():
    if baro is not None:
        query = "LOCATIONID = {:}".format(baro)
        baro_out[baro] = ll.table_to_pandas_dataframe(gw_reading_table,query=query)
        print(baro)


KeyboardInterrupt: 

In [ ]:
well_table['BaroLoggerType'].unique()

In [ ]:
j=0
welly = {}

welliddict = {'7002':'D3-41','7001':'F3-06'} 
wellids = ['7002','7001']
beff,wells = {},{}
for wellid in wellids:
    for file in glob.glob("G:/My Drive/WORK/Juab/WellData/*{:}}*".format(welliddict[wellid])):
        print(file)
        well = ll.new_trans_imp(file)
        corrwl = ll.well_baro_merge(well, baro_out['9062'], barocolumn='MEASUREDLEVEL', vented=False)

        wls, be = correct_be(wellid, well_table, corrwl,be=0.72)
        stdata = well_table.loc[wellid,:]
        man_sub = manual[manual['Location ID'] == int(wellid)]
        well_elev = float(stdata['Altitude'])    
        stickup = float(stdata['Offset'])
        man_sub.loc[:, 'MeasuredDTW'] = man_sub['Water Level (ft)'] * -1
        man_sub.loc[:, 'Meas_GW_Elev'] = man_sub['MeasuredDTW'].apply(lambda x: float(well_elev) + (x + float(stickup)),
                                                                          1)
        print('Stickup: {:}, Well Elev: {:}'.format(stickup, well_elev))

        # fix transducer drift

        dft = ll.fix_drift(wls, man_sub, meas='BAROEFFICIENCYLEVEL', manmeas='MeasuredDTW')
        drift = np.round(float(dft[1]['drift'].values[0]), 3)

        df = dft[0]
        df.sort_index(inplace=True)
        first_index = df.first_valid_index()
        rowlist, fieldnames = ll.prepare_fieldnames(df, wellid, stickup, well_elev)
        #beff[j] = m
        wells[j] = rowlist 
        j += 1
    welly[wellid] = pd.concat(wells)

In [ ]:
alldata = pd.concat(wells)
alldata.drop('level_0',inplace=True,axis=1)
alldata.reset_index(inplace=True)
alldata.set_index('READINGDATE',inplace=True)
alldata = alldata[alldata.index > pd.datetime(2015,5,15)]
alldata = alldata[alldata['WATERELEVATION']>5145]
alldata['WATERELEVATION'] = alldata['WATERELEVATION'].apply(lambda x: round(x,3),1)
alldata.drop(['level_0','level_1','Level','corrwl','name','julian','datechange'],axis=1,inplace=True)
alldata.drop(['barometer','dbp','dwl'],axis=1,inplace=True)
alldata.sort_index(inplace=True)
alldata.drop_duplicates(inplace=True)

manual_file = 'G:/My Drive/WORK/Snake Valley/manual_measurements.csv'
manual = pd.read_csv(manual_file, index_col="DateTime",parse_dates=True)
manual = manual[manual['Location ID']==7001]
manual['wlelev'] = well_elev - (manual['Water Level (ft)'] - stickup)
plt.plot(alldata.index, alldata['WATERELEVATION'],zorder=0)
plt.scatter(manual.index, manual['wlelev'], color='red', zorder=1)

In [26]:
cols = ['SITEID','DATETIME','A','B','C','D','Install Ground Elevation','STICKUP','CAP','DTW Current (ft)','WL_ELEV']


In [27]:
copydir = "G:/My Drive/WORK/Snake Valley/CorrectedWetlandFiles/"

j = 0

writer = pd.ExcelWriter('G:/My Drive/WORK/Snake Valley/CorrectedWetlandFiles/compiledfield3.xlsx')


for xlfile in glob.glob(copydir+'/*'):
    print(xlfile)
    try:
        df = pd.read_excel(xlfile,'PreviousDownloadLastLine',names=cols)
        df.to_excel(writer, "A"+str(j))
        writer.save()
        j += 1
    except:
        pass

G:/My Drive/WORK/Snake Valley/CorrectedWetlandFiles\2017-01-18_BARO1PiezometersCorrected2010_1.xlsx
G:/My Drive/WORK/Snake Valley/CorrectedWetlandFiles\2012-06-19_BARO2PiezometersCorrected2010_1.xlsx
G:/My Drive/WORK/Snake Valley/CorrectedWetlandFiles\2012-06-19_Baro3PiezometersCorrected2010_1.xlsx
G:/My Drive/WORK/Snake Valley/CorrectedWetlandFiles\2012-06-21_BARO1PiezometersCorrected2010_2.xlsx
G:/My Drive/WORK/Snake Valley/CorrectedWetlandFiles\2012-06-21_BARO2PiezometersCorrected.xlsx
G:/My Drive/WORK/Snake Valley/CorrectedWetlandFiles\2012-06-20_BARO3PiezometersCorrected_2010_2.xlsx
G:/My Drive/WORK/Snake Valley/CorrectedWetlandFiles\2012-06-27_BARO3PiezometersCorrected1051.xlsx
G:/My Drive/WORK/Snake Valley/CorrectedWetlandFiles\2012-06-29_BARO1PiezometersCorrected_2010_3.xlsx
G:/My Drive/WORK/Snake Valley/CorrectedWetlandFiles\2012-06-21_BARO2PiezometersCorrected_2010_3.xlsx
G:/My Drive/WORK/Snake Valley/CorrectedWetlandFiles\2012-06-21_BARO3PiezometersCorrected_2010_3.xlsx
G:/M